In [1]:
#install.packages("magrittr")
#install.packages("purrr")
#install.packages("glue")
#install.packages("stringr")
#install.packages("rvest")
#install.packages("polite")
#install.packages("tidyverse")
#install.packages("qdapRegex")
library(qdapRegex)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(tidyverse)
library(ggplot2)
library(lubridate)
library(dplyr)


Attaching package: ‘purrr’


The following object is masked from ‘package:magrittr’:

    set_names


Loading required package: xml2


Attaching package: ‘rvest’


The following object is masked from ‘package:purrr’:

    pluck


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2          ✔ readr   1.3.1     
✔ tibble  3.0.3          ✔ dplyr   1.0.2.9000
✔ tidyr   1.1.2          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::%+%()          masks qdapRegex::%+%()
✖ dplyr::collapse()       masks glue::collapse()
✖ dplyr::explain()        masks qdapRegex::explain()
✖ tidyr::extract()        masks magrittr::extract()
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()
✖ rvest::pluck()          masks purrr::pluck()
✖ purrr::set_names()      masks magrittr::set_names()


Attaching pa

In [2]:
# Creating a function to scrape the tables in Wikipedia
get_data <- function(year){
    Link <- glue("https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States_in_{year}")
    page_title <- read_html(Link)
    Date <- page_title %>% html_nodes(".wikitable.sortable") %>% html_nodes("span") %>% html_text()
    n <- Date %>% as.matrix() %>% nrow()
    a <- page_title %>% html_nodes(".wikitable.sortable") %>% html_nodes("td") %>% html_text() %>% as.matrix()
    Date <- as.vector('')
    Location <- as.vector('')
    Injured <- as.vector('')
    Death <- as.vector('')
    Total <- as.vector('')
    Des <- as.vector('')
    ## split matrix into different vector
    for( i in 1:nrow(as.matrix(a))){
        if (i %% 6 == 1){
            Date[(i%/%6) +1] = a[i]
        } else{
        if (i %% 6 == 2){
            Location[(i%/%6 +1)] = a[i]
        } else{
        if (i %% 6 == 3){
            Death[(i%/%6)+1] = a[i]
        } else{
        if (i %% 6 == 4){
            Injured[(i%/%6)+1] = a[i]
        } else{
        if (i %% 6 == 5){
            Total[(i%/%6)+1] = a[i]
        } else{
            Des[(i%/%6)] = a[i]
        } 
        }
        }        
        }
        }        
    }
    Injured <- Injured %>% rm_between(.,"[","]")
    Death <- Death %>% rm_between(.,"[","]")
    Total <- Total %>% rm_between(.,"[","]")
    Des <- Des %>% rm_between(.,"[","]")
    Injured <- str_remove(Injured,"[+]")
    Death <- Death %>% str_remove(.,"[+]")
    Total <- Total %>% str_remove(.,"[+]")
    df <-data.frame(Date = mdy(Date[1:n]))
    df$Location <- Location[1:n] 
    df$Death <- as.integer(Death[1:n])
    df$Injured <- as.integer(Injured[1:n])
    df$Total <- as.integer(Total[1:n])
    df$Description <- Des[1:n]
    df_final <- df %>% filter(year(df$Date)==year)   
    return(df_final)
}


    



In [3]:
# Writing files into csv
year <- c("2018","2019","2020")

for(i in 1:nrow(as.matrix(year))){
    data <- get_data(year[i])
    write.csv (data,glue("{year[i]}.csv"))   
}


Warning message:
“ 5 failed to parse.”
